In [1]:
import tensorflow as tf

2024-03-08 21:35:10.087296: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-08 21:35:10.114945: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-08 21:35:11.385215: I itex/core/wrapper/itex_cpu_wrapper.cc:42] Intel Extension for Tensorflow* AVX512 CPU backend is loaded.
2024-03-08 21:35:11.959962: I itex/core/wrapper/itex_gpu_wrapper.cc:35] Intel Extension for Tensorflow* GPU backend is loaded.
2024-03-08 21:35:11.992465: W itex/core/op

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define paths to your training and testing directories
train_dir = 'model_dataset/train'
test_dir = 'model_dataset/test'

# Define ImageDataGenerator for training data
train_datagen = ImageDataGenerator(rescale=1./255) # rescale pixel values to [0,1]

# Define ImageDataGenerator for testing data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=32,
        class_mode='binary')  # Since we use binary_crossentropy loss, we need binary labels

# Flow validation images in batches of 32 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

# Define your model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)


Found 3245 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


2024-03-08 21:35:12.569383: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform XPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-08 21:35:12.569424: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:XPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: XPU, pci bus id: <undefined>)
2024-03-08 21:35:13.307692: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type CPU is enabled.


Epoch 1/100
101/101 [==============================] - 6s 46ms/step - loss: 0.2065 - accuracy: 0.9374 - val_loss: 0.7057 - val_accuracy: 0.8017
Epoch 2/100
101/101 [==============================] - 2s 19ms/step - loss: 0.1066 - accuracy: 0.9716 - val_loss: 0.4257 - val_accuracy: 0.8667
Epoch 3/100
101/101 [==============================] - 2s 17ms/step - loss: 0.0732 - accuracy: 0.9797 - val_loss: 0.3674 - val_accuracy: 0.8900
Epoch 4/100
101/101 [==============================] - 2s 18ms/step - loss: 0.0594 - accuracy: 0.9818 - val_loss: 0.3819 - val_accuracy: 0.8817
Epoch 5/100
101/101 [==============================] - 2s 16ms/step - loss: 0.0653 - accuracy: 0.9849 - val_loss: 0.4070 - val_accuracy: 0.8567
Epoch 6/100
101/101 [==============================] - 2s 18ms/step - loss: 0.0420 - accuracy: 0.9877 - val_loss: 0.5298 - val_accuracy: 0.8567
Epoch 7/100
101/101 [==============================] - 2s 19ms/step - loss: 0.0298 - accuracy: 0.9911 - val_loss: 0.3847 - val_accuracy:

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 17, 17, 128)       0

In [16]:




import numpy as np
from tensorflow.keras.preprocessing import image


# Define a function to load and preprocess an image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Path to the image you want to predict
image_path = 'my_test.jpg'

# Preprocess the image
preprocessed_image = preprocess_image(image_path)

# Make predictions
predictions = model.predict(preprocessed_image)

# Class 1 or 0 depending on the threshold
threshold = 0.5
class_label = "non-human" if predictions[0][0] < threshold else "human"

print("Predictions:", predictions)
print("Class:", class_label)


1/1 [==============================] - 0s 22ms/step
Predictions: [[0.]]
Class: non-human


In [14]:
image_path = 'Screenshot 2024-01-24 102521.png'
preprocessed_image = preprocess_image(image_path)
predictions = model.predict(preprocessed_image)

threshold = 0.5
class_label = "non-human" if predictions[0][0] < threshold else "human"

print("Predictions:", predictions)
print("Class:", class_label)


1/1 [==============================] - 0s 21ms/step
Predictions: [[1.]]
Class: human
